# Summary

This script is to processing the mass spectral embeddings into arrays and connect them to the class labels.



In [1]:
import os
import gensim
import pickle
import time
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from spec2vec.vector_operations import calc_vector
from collections import OrderedDict, Counter

## Import data from file

In [2]:
data_path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings/datasets/ALL_GNPS_210409_positive/"
base = "ALL_GNPS_210409_positive_cleaned"
spectra_data = "_peaks_processed_s2v_only_annotated.pickle"

embedding_path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings/embeddings/ALL_GNPS_210409_positive/ALL_GNPS_210409_positive_cleaned_spec2vec_embedding_iter_15.model"
save_path = "/mnt/scratch/ding013/MS2ChemClass/hooft_data/embedding_visualization_out/"

spectrum_file = os.path.join(data_path, base+".pickle")
os.path.exists(spectrum_file)

processed_spectrums_file = os.path.join(data_path, base+spectra_data)
with open(processed_spectrums_file, 'rb') as inf:
    spectrums_processed = pickle.load(inf)

len(spectrums_processed)

# TEST DATA
# spectrums_processed = spectrums_processed[1:100]
# len(spectrums_processed)


# ### Keep annotated spectra
# Annotated with inchikey

annot_spectrums_processed = []
for spec in spectrums_processed:
    inchikey = spec.metadata.get("inchikey")
    if inchikey:
        annot_spectrums_processed.append(spec)
len(annot_spectrums_processed)

spectrums_ids = [s.metadata.get("spectrum_id") for s in annot_spectrums_processed]


any([s==None for s in spectrums_ids])

False

## Create mass spectral embedding vectors

In [3]:
# ### Create spectral embedding vectors


# import model
model_file = os.path.join(embedding_path)
model = gensim.models.Word2Vec.load(model_file)


from spec2vec import SpectrumDocument

documents_spectrums_processed = [SpectrumDocument(s, n_decimals=2) for s in annot_spectrums_processed]
print(len(documents_spectrums_processed))


spectrum_vectors= [calc_vector(model, s, intensity_weighting_power=0.5) for s in documents_spectrums_processed]

spectrum_vectors = np.vstack(spectrum_vectors)
spectrum_vectors.shape

# and for UniqueInchikey
sys.path.insert(0, os.path.join(os.path.split(os.getcwd()[:-1])[0], "scripts"))

from mass_differences.processing import get_ids_for_unique_inchikeys ## This function is cited from Joris Louwen
uniq_ids = get_ids_for_unique_inchikeys(annot_spectrums_processed)


spectrum_vectors_unique = [spectrum_vectors[i] for i in uniq_ids]
spectrum_vectors_unique = np.vstack(spectrum_vectors_unique)
spectrum_vectors_unique.shape

187152


(16360, 300)

## Link classification labels to embeddings

In [4]:
# ### Link classification labels to selected spectra

classifications_path = "/mnt/LTR_userdata/hooft001/mass_spectral_embeddings/classifications/ALL_GNPS_210409_positive/"
classes_file = os.path.join(classifications_path, "ALL_GNPS_210409_positive_processed_annotated_CF_NPC_classes.txt")
classes = {}  # inchikey: [classes] in order of header
with open(classes_file) as inf:
    header = inf.readline().strip().split('\t')
    for line in inf:
        line = line.strip().split('\t')
        classes[line[0]] = line[2:]
print(f'read classes for {len(classes)} inchikeys')
print("\nclasses order:")
print(header)

spectra_classes = []
empty_class = ["" for _ in range(len(header)-2)]
assert len(empty_class) == len(header)-2
for s_id, spec in zip(spectrums_ids, annot_spectrums_processed):
    inchikey = spec.metadata.get("inchikey")
    s_classes = classes.get(inchikey, empty_class)
    assert len(s_classes) == len(header)-2
    res = [s_id] + s_classes
    spectra_classes.append(res)
print(len(spectra_classes))
print(spectra_classes[:2])

new_header = ['spectrum_id'] + header[2:]
assert len(new_header) == len(spectra_classes[0])
print(new_header)

read classes for 19824 inchikeys

classes order:
['inchi_key', 'smiles', 'cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass', 'cf_direct_parent', 'npc_class_results', 'npc_superclass_results', 'npc_pathway_results', 'npc_isglycoside']
187152
[['CCMSLIB00000001547', 'Organic compounds', 'Organic acids and derivatives', 'Peptidomimetics', 'Hybrid peptides', 'Hybrid peptides', 'Cyclic peptides; Microcystins', 'Oligopeptides', 'Amino acids and Peptides', '0'], ['CCMSLIB00000001548', 'Organic compounds', 'Organic acids and derivatives', 'Peptidomimetics', 'Depsipeptides', 'Cyclic depsipeptides', 'Cyclic peptides', 'Oligopeptides', 'Amino acids and Peptides', '0']]
['spectrum_id', 'cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass', 'cf_direct_parent', 'npc_class_results', 'npc_superclass_results', 'npc_pathway_results', 'npc_isglycoside']


In [5]:
spectra_classes[0]

['CCMSLIB00000001547',
 'Organic compounds',
 'Organic acids and derivatives',
 'Peptidomimetics',
 'Hybrid peptides',
 'Hybrid peptides',
 'Cyclic peptides; Microcystins',
 'Oligopeptides',
 'Amino acids and Peptides',
 '0']

### Get the ordered classification labels

In [6]:
# i_chosen = -4  # npc_class # choose which class (npc/cf) to plot (in new_header)
# i_chosen = 3 # cf_class

## Collect the ordered labels in all classes
all_classes_ori = []
for lbs in range(1,len(new_header)):
    print(new_header[lbs])
    chosen_classes_ori = []
    uncl = "Unclassified"
    uncl_other = "Other/Unclassified"
    add_uncl = False
    for s_cl in spectra_classes:
        chosen_class = s_cl[lbs]
        if not chosen_class:
            chosen_class = uncl
        else:  # separate in single classes - there are compounds with 3 superclasses for example
            chosen_class = chosen_class.split("; ")[0]
        chosen_classes_ori.append(chosen_class)
    len(chosen_classes_ori)
    all_classes_ori.append(chosen_classes_ori)
len(all_classes_ori)

cf_kingdom
cf_superclass
cf_class
cf_subclass
cf_direct_parent
npc_class_results
npc_superclass_results
npc_pathway_results
npc_isglycoside


9

In [7]:
## Collect the orderen labels with unique inchikey in all classes
all_classes_ori_ui = []
for k in all_classes_ori:
    chosen_classes_ori_ui = [k[i] for i in uniq_ids]
    len(chosen_classes_ori_ui)
    all_classes_ori_ui.append(chosen_classes_ori_ui)
len(all_classes_ori_ui)

9

## Get the belonging relationship among labels in differnt class levels

In [17]:
cf_labels = all_classes_ori_ui[1:4] # Superclass, Class, Subclass
npc_labels = all_classes_ori_ui[5:8] # Class, SuperClass, Pathway

In [67]:
def get_belonging_relationship(list1, list2):
    lbs_dict={}
    for i in set(list1):
        lbs_list = []
        for j in range(len(list2)):
            if list1[j] == i:
                lbs_list.append(list2[j])
        lbs_dict[i] = list(set(lbs_list))
    return lbs_dict

def combine_two_belonging_dict(dict1,dict2):
    list1=[]
    list2=[]
    list3=[]
    for i,j in dict1.items():
        for k,v in dict2.items():
            list2 += [k]*len(v)
            list3 += v
            list1 += [i]*len(v)
    combine_list = [list1,list2,list3]
    return combine_list

In [85]:
# for ClassyFire
cf_superclass_class = get_belonging_relationship(cf_labels[0],cf_labels[1])
cf_class_subclass = get_belonging_relationship(cf_labels[1], cf_labels[2])
cf_lbs_belong_list = combine_two_belonging_dict(cf_superclass_class, cf_class_subclass)
np.savetxt('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/cf_label_belong.csv', np.array(cf_lbs_belong_list), delimiter=',',fmt='%s')

# for NPClassifier
npc_superclass_class = get_belonging_relationship(npc_labels[1],npc_labels[0])
npc_class_pathway = get_belonging_relationship(npc_labels[0],npc_labels[2])
npc_lbs_belong_list = combine_two_belonging_dict(npc_superclass_class, npc_class_pathway)
np.savetxt('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/npc_label_belong.csv', np.array(npc_lbs_belong_list), delimiter=',',fmt='%s')


asd


In [81]:
print(len(cf_lbs_belong_list[0]))
print(len(cf_lbs_belong_list[1]))
print(len(cf_lbs_belong_list[2]))
print(cf_lbs_belong_list[2][1])

print(len(npc_lbs_belong_list[0]))
print(len(npc_lbs_belong_list[1]))
print(len(npc_lbs_belong_list[2]))
print(npc_lbs_belong_list[2][2])

13300
13300
13300
1,2,4-triazolopyrimidine-2-sulfonamides
35145
35145
35145
Fatty acids


## Save array as csv file

In [8]:
## Save the spectral embeddings array with unique inchikey

X_embedding = np.array(spectrum_vectors_unique)
np.savetxt('/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/X_spectral_embeddings.csv', X_embedding, delimiter=',')

In [9]:
## Save the ordered class labels array with unique inchikey
for i in range(len(all_classes_ori_ui)):
    y_label = np.array(all_classes_ori_ui[i])
    print(new_header[i+1])
    filepath_labels='/mnt/scratch/ding013/MS2ChemClass/embedding_arrays_for_classification/y_%s_labels.csv' % new_header[i+1]
    np.savetxt(filepath_labels, y_label, delimiter=',',fmt='%s')

cf_kingdom
cf_superclass
cf_class
cf_subclass
cf_direct_parent
npc_class_results
npc_superclass_results
npc_pathway_results
npc_isglycoside
